## Numeric Data

The API serves normalized and as-reported financial information in JSON.

Calcbench uses cookie based authentication, you will need to use a client that handles cookies, an example using Python's requests package is below.

In [1]:
import requests
import json
cb_email = "andrew@calcbench.com" # put your calcbench username here
cb_password = "password" # put your calcbench password here
CALCBENCH_SESSION = requests.Session()
CALCBENCH_API_URL_BASE = "https://www.calcbench.com/api/{0}"
r = CALCBENCH_SESSION.post('https://www.calcbench.com/account/LogOnAjax', 
                  {'email' : cb_email, 
                   'strng' : cb_password, 
                   'rememberMe' : 'true'}, 
                  verify=False)
assert r.text == 'true', 'login failed'

#### Get tickers by industry.

In [2]:
pharma_SIC_code = 2834
url = CALCBENCH_API_URL_BASE.format("companies?siccodes={0}".format(pharma_SIC_code))
pharma_companies = CALCBENCH_SESSION.get(url).json()
pharma_tickers = [co['ticker'] for co in pharma_companies]

### Normalized Data

In [4]:
metrics = ['revenue', 'netincome', 'assets', 'stockholdersequity']
payload = {"start_year" : 2010,
           'start_period' : 1,
           'end_year' : 2014,
           'end_period' : 4,
           'company_identifiers' : ['ibm', 'goog'],
           'metrics' : metrics,
           }
url = CALCBENCH_API_URL_BASE.format("/NormalizedValues")
data = CALCBENCH_SESSION.post(url,
                              data=json.dumps(payload), 
                              headers={'content-type' : 'application/json'}
                              ).json()

In [12]:
data[0]

{'calendar_period': 1,
 'calendar_year': 2010,
 'metric': 'revenue',
 'ticker': 'GOOG',
 'value': 6775000000.0}

### As Reported Data

In [3]:
ticker = "ibm"
statement_type = "income" # one of (income, balance, cash)
as_reported_path = 'asreported/?companyIdentifier={0}&statementType={1}&periodType=annual'.format(ticker, statement_type)
response = CALCBENCH_SESSION.get(CALCBENCH_API_URL_BASE.format(as_reported_path), verify=True)
as_reported_data = json.loads(response.text)

In [4]:
as_reported_data

{'columns': [{'fiscal_period': 'Y 2014',
   'instant': False,
   'period_end': '2014-12-31',
   'period_start': '2014-01-01'},
  {'fiscal_period': 'Y 2013',
   'instant': False,
   'period_end': '2013-12-31',
   'period_start': '2013-01-01'},
  {'fiscal_period': 'Y 2012',
   'instant': False,
   'period_end': '2012-12-31',
   'period_start': '2012-01-01'},
  {'fiscal_period': 'Y 2011',
   'instant': False,
   'period_end': '2011-12-31',
   'period_start': '2011-01-01'}],
 'entity_name': 'International Business Machines Corp',
 'line_items': [{'is_subtotal': False,
   'label': 'Revenue',
   'local_name': 'RevenuesAbstract',
   'tree_depth': 1},
  {'facts': [{'effective_value': 55673000000.0,
     'fact_id': 87677714,
     'format_type': 'currency',
     'unit_of_measure': 'USD'},
    {'effective_value': 57655000000.0,
     'fact_id': 87677950,
     'format_type': 'currency',
     'unit_of_measure': 'USD'},
    {'effective_value': 59453000000.0,
     'fact_id': 87677951,
     'format_typ